# Zaawansowane operacje w SQL - stringi, daty, aliasy, joins i podzapytania

_Mikołaj Leszczuk_

## Konspekt

* Operacje na stringach
* Funkcje daty
* Aliasy - zastępcze nazwy kolumn lub tabeli
* Łączenie tabel i wyszukiwanie danych w wielu tabelach
* Pod-zapytania

## Operacje na stringach

Tu doświadczymy istotnych różnic pomiędzy implementacjami DBMS!

In [ ]:
!cp simple_library_original.db simple_library.db
%load_ext sql
%sql sqlite:///simple_library.db?mode=rw

In [ ]:
%%sql
SELECT * FROM simple_library;

### `||`

Czasem jest potrzeba wyświetlenia konkatenacji kolumn i jakiegoś łańcucha tekstowego. Do tego może przydać się operator `||`.

Sposób realizacji konkatenacji różni się od siebie w bazach danych:

* SQLite: `||`
* Oracle: `||` lub `CONCAT()` 
* MySQL: `CONCAT()`
* SQL Server: `+`

Składnia użycia operatora `||` w SQLite/Oracle:

```sql
SELECT nazwa_kolumny1 || 'dowolny_tekst' || nazwa_kolumny2
FROM nazwa_tabeli;
```

##### Przykład

> **Przykład:**
>
>  Użycie `||` w SQLite/Oracle. **Wyświetl unikalne imię i nazwisko oddzielone spacją `(" ")`**

In [ ]:
%%sql
SELECT Author_name || ' ' || Author_surname FROM simple_library;

## Unikalna lista autorów

### Ćwiczenie

Wyświetl bez powtórzeń imię i nazwisko oddzielone spacją `(' ')`.

### Rozwiązanie

In [ ]:
%%sql
SELECT DISTINCT Author_name || ' ' || Author_surname FROM simple_library;

### `LOWER()` i `UPPER()`

Funkcja zwraca wartość kolumny tekstowej w postaci małych/dużych liter.

Składnia:

```sqlite
SELECT LOWER(nazwa_kolumny) FROM nazwa_tabeli;
```

```sqlite
SELECT UPPER(nazwa_kolumny) FROM nazwa_tabeli;
```

> **Przykład:**
> 
> Wyświetl przy pomocy LOWER() wszystkie nazwiska małymi literami.

In [ ]:
%%sql
SELECT LOWER(Author_surname) FROM simple_library;

> **Przykład:**
> 
> Wyświetl przy pomocy UPPER() wszystkie imiona dużymi literami.

In [ ]:
%%sql
SELECT UPPER(Author_name) FROM simple_library;

## Imię małymi, nazwisko wielkimi literami

### Ćwiczenie

Wyświetl bez powtórzeń imię małymi literami i nazwisko dużymi literami.

### Rozwiązanie

In [ ]:
%%sql
SELECT DISTINCT LOWER(author_name), UPPER(author_surname) FROM simple_library;

### `LENGTH()`

Funkcja zwraca długość pola tekstowego podanego jako argument

Składnia:

```sqlite
SELECT LENGTH(nazwa_kolumny) FROM nazwa_tabeli;
```

W SQL Server używa się do tego samego funkcji `LEN()`

> **Przykład:**
>
> Wyświetl przy pomocy LENGTH() nazwisko i odpowiadającą jemu długość nazwiska.

In [ ]:
%%sql
SELECT Author_surname, LENGTH(Author_surname) FROM simple_library;

### `REPLACE()`

Funkcja przeszukuje `str1` w poszukiwaniu `str2` aby go zamienić na `str3`

Składnia:

```sqlite
SELECT REPLACE(nazwa_kolumny,'str2','str3') FROM nazwa_tabeli;
```

> **Przykład:**
>
> Wyświetl imiona i nazwiska z tym, że za pomocą `REPLACE()` zamiast imienia `'Stanisław'` wyświetl `'Stan'`.

In [ ]:
%%sql
SELECT REPLACE(Author_name, 'Stanisław', 'Stan'), Author_surname FROM simple_library;

### `SUBSTR()`

Funkcja ta służy do operacji na łańcuchach tekstowych w wyniku zapytań. Może przyjmować 2-3 argumentów.

```sql
SUBSTR(nazwa_kolumny, pozycja [, liczba_znaków])
```

`nazwa_kolumny` - łańcuch tekstowy do obcięcia

`pozycja` - nr znaku od którego ma być wyświetlony string

`liczba_znaków` - argument opcjonalny, liczba znaków do wyświetlenia, liczona od parametru pozycja

Składnia:

```sqlite
SELECT SUBSTR(nazwa_kolumny,pozycja[,liczba_znaków])
FROM nazwa_tabeli;
```

W Oracle oraz w niektórych wersjach MySQL używa się funkcji `SUBSTRING()`. Natomiast w MS Access stosuje się `MID()`.

> **Przykład:**
>
> Wyświetl przy pomocy `SUBSTR()` dwie pierwsze litery z nazwiska.

In [ ]:
%%sql
SELECT SUBSTR(Author_surname, 1, 2) FROM simple_library;

## Funkcje daty

### Funkcje daty SQLite

Umożliwiają operacje na dacie i czasie w trakcie wykonywania zapytania. Typowe funkcje to:

`strftime('%d')`, `strftime('%m')`, `strftime('%Y')`

Zwracają kolejno dzień, miesiąc, rok z podanej w argumencie daty

```sqlite
SELECT strftime('%d', data);
```

> **Przykład:**
> 
> Wybieramy książki, które zarejestrowano lutym.

In [ ]:
%%sql
SELECT * FROM simple_library WHERE STRFTIME('%m', Registration_date)='02';

## Wyszukiwanie książek dodanych na początku miesiąca

#### Ćwiczenie

Wybierz książki, których dzień dodania należy do przedziału od `01` do `10`.

### Rozwiązanie

In [ ]:
%%sql
SELECT * FROM simple_library WHERE STRFTIME('%d', Registration_date) BETWEEN '01' AND '10';

### `DATE()`

W SQLite funkcja DATE() służy do manipulacji i formatowania dat.

Naczęstsze zastosowane to `DATE('now')`, które zwraca bieżącą datę i czas.

> **Przykład:**
>
> Zwracanie aktualnej datę w formacie YYYY-MM-DD na podstawie czasu UTC.

In [ ]:
%%sql
SELECT DATE('now');

### `JULIANDAY()`

Funkcja `JULIANDAY()` zwraca liczbę dni, które upłyneły od od **4713 roku p.n.e. (12:00 UTC, 1 stycznia)** według **kalendarza juliańskiego**. Jest to standard używany w astronomii do mierzenia czasu w sposób ciągły.

**Dlaczego 4713 rok p.n.e.?**

Rok ten został wybrany przez Josepha Scaligera w 1583 roku jako początek ery juliańskiej, co pozwala na unikanie problemów związanych z różnymi kalendarzami i epokami historycznymi. System ten umożliwia łatwe obliczanie różnic między datami.

> **Przykład:**
>
> Zwrócenie aktualnej daty w formacie juliańskim.

In [ ]:
%%sql
SELECT JULIANDAY('now');

## Oblicz liczbę dni między dwiema ważnymi datami w historii Polski

### Ćwiczenie

Oblicz liczbę dni, które upłynęły między **odzyskaniem niepodległości przez Polskę (11 listopada 1918)** a **wejściem Polski do Unii Europejskiej (1 maja 2004)**.

**Podpowiedź:** Aby uzyskać liczbę dni, odejmij w zapytaniu wartość `JULIANDAY()` dla starszej daty od wartości `JULIANDAY()` dla nowszej daty.

### Rozwiązanie

In [ ]:
%%sql
SELECT JULIANDAY('2004-05-01') - JULIANDAY('1918-11-11');

## Aliasy - zastępcze nazwy kolumn lub tabeli

Alias to nazwa zastępcza, którą możemy zdefiniować dla konkretnej kolumny lub tabeli na początku zapytania, a następnie używać w całym zapytaniu jako wygodniejsza (zazwyczaj dużo krótsza) nazwa.

Alias definiuje się używając klauzuli “`AS`” zaraz po nazwie kolumny lub tabeli w następujący sposób:

```sqlite
SELECT Kolumna AS Kol FROM tabela AS tab WHERE Kol = wartość;
```

Od momentu takiej definicji do kolumny kolumna wystarczy odwoływać się aliasem `Kol` a tabeli tabela aliasem `tab`.

## Łączenie tabel i wyszukiwanie danych w wielu tabelach

Czasem potrzebujemy wybrać dane z tabeli w zależności od danych pochodzących z innej tabeli. Do rozwiązania tego problemu służy operacja zwana **łączeniem (`JOIN`)**.

Jeśli w zapytaniu potrzebne nam są dane z tabel: `tabela1` i `tabela2`, to musimy wpisać ich nazwy zaraz po słowie `FROM` (tak jak to robimy w standardowych zapytaniach) a następnie odwoływać się do kolumn z tych tabel używając: nazwy konkretnej tabeli, operatora kropki “`.`” oraz nazwy żądanej kolumny.

Do kolumny w konkretnej tabeli odwołujemy się za pomocą operatora kropki “`.`”. Jest to szczególnie istotne w zapytaniach, w których korzysta się z więcej niż jednej tabeli.

```sqlite
SELECT tabela1.Kolumna1, tabela2.Kolumna2 FROM tabela1, tabela2
ON tabela1.Kolumna1 = tabela2.Kolumna2;
```

Operator “`,`” jest skrótem od słowa kluczowego `JOIN`, które oznacza zebranie wymienionych tabel w jedną dużą i operowanie na niej.

Możliwe jest łączenie trzech i więcej tabel. Działa to analogicznie jak w przypadku dwóch tabel.

`JOIN` łączy rekordy z dwóch+ kolumn w jeden na podstawie podanego warunku/podanych warunków.

In [ ]:
!cp library_original.db library.db
%load_ext sql
%sql sqlite:///library.db?mode=rw

Schemat bazy danych z pliku `library.db`:

![](library.png)

In [ ]:
%%sql
SELECT * FROM authors;

In [ ]:
%%sql
SELECT * FROM titles;

In [ ]:
%%sql
SELECT * FROM books;

In [ ]:
%%sql
SELECT * FROM borrowings;

In [ ]:
%%sql
SELECT * FROM users;

In [ ]:
%%sql
SELECT * FROM fines;

> **Przykład:**
>
> Łączymy rekordy z tabel `authors` i `titles` w jedną tabelę na podstawie podanego warunku równości `authors.Id` i `titles.Author_ID`.

In [ ]:
%%sql
SELECT * FROM authors JOIN titles ON authors.ID = titles.Author_ID;

## Wyszukiwanie książek konkretnego autora

### Ćwiczenie

Znajdź nazwisko i tytuły książek napisanych przez autora o identyfikatorze `'3'`.

### Rozwiązanie

In [ ]:
%%sql
SELECT authors.Surname, titles.Title FROM authors JOIN titles ON authors.ID = titles.Author_ID
WHERE titles.Author_ID = '3';

## Wyszukiwanie książek autorów o imieniu na literę `‘S’`

### Ćwiczenie

Znajdź imiona, nazwiska i tytuły książek napisanych przez autorów o imieniu rozpoczynającym się na `'S'`.

### Rozwiązanie

In [ ]:
%%sql
SELECT authors.Name, authors.Surname, titles.Title FROM authors JOIN titles
ON authors.ID = titles.Author_ID WHERE authors.Name LIKE 'S%';

**Możliwe jest łączenie tabel z użyciem aliasów**

Aliasy znajdują praktyczne zastosowanie w bardziej złożonych zapytaniach np. korzystających ze złączenia tabel czy podzapytania.

```sql
SELECT tabela1.Kolumna1 FROM Tabela1 AS tab1, tabela2 AS tab2 WHERE tab2.kolumna=wartość;
```

> **Przykład:**
> 
> Podajemy id tytułu, nazwisko autora i tytuł autora o ID `1` korzystając z aliasów dla tabel `titles` i `authors`.

In [ ]:
%%sql
SELECT t.ID, a.Surname, t.Title FROM authors AS a, titles AS t ON a.ID = t.Author_ID WHERE a.ID = '1';

## Pod-zapytania

Tworzenie podzapytania polega na zagnieżdżaniu zapytania w innym zapytaniu.

Aby je zagnieździć, wystarczy w miejscu, w którym oczekujemy zwróconej przez podzapytanie wartości, wstawić nawiasy i zapisać pomiędzy nimi zapytanie w takiej samej formie jak zwykłe zapytanie.

```sqlite
SELECT Kolumna1, ... FROM tabela1
WHERE Kolumna1=(SELECT Kolumna2 FROM tabela2);
```

> **Przykład:**
>
> Wybieramy id, imię i nazwisko autora który napisał książkę o tytule `'Solaris'`.

In [ ]:
%%sql
SELECT Author_ID FROM titles WHERE Title = 'Solaris';

In [ ]:
%%sql
SELECT ID, Name, Surname FROM authors WHERE ID = '1';

In [ ]:
%%sql
SELECT ID, Name, Surname FROM authors WHERE ID = (
    SELECT Author_ID FROM titles WHERE Title = 'Solaris'
);

## Wyszukiwanie książek Lema za pomocą podzapytania

### Ćwiczenie

Wybierz przy pomocy podzapytania tytuły wszystkich książek Lema.

### Rozwiązanie

In [ ]:
%%sql
SELECT Title FROM titles WHERE Author_ID IN (
    SELECT ID FROM authors WHERE Surname = 'Lem'
);

Wynik każdego zapytania `SELECT` można traktować jak kolejną (wirtualną) tabelę. W efekcie możemy wykonywać na takiej tabeli zapytania.

* Pod-zapytania w `FROM`
* Pod-zapytania w `WHERE`
* Pod-zapytania w `SELECT`

### Podzapytania w `FROM`

> Przykład
> 
> Załóżmy, że mamy gotowe zapytanie o listę użytkowników biblioteki wraz z liczbą książek pożyczonych przez każdego z nich.
>
> Na wyniku tego zapytania możemy wykonać kolejne zapytanie, by otrzymać listę użytkowników, którzy pożyczyli więcej niż 10 książek.

In [ ]:
%%sql
SELECT users.Name, users.Surname, COUNT(borrowings.ID) AS Activity
FROM users JOIN borrowings ON users.ID = borrowings.User_ID GROUP BY users.ID;

In [ ]:
%%sql
SELECT * FROM
(
    SELECT users.Name, users.Surname, COUNT(borrowings.ID) AS Activity
    FROM users JOIN borrowings ON users.ID = borrowings.User_ID GROUP BY users.ID
)
WHERE Activity>10;

### Podzapytania w `WHERE`

> Przykład
> 
> Zapytania zwracające w wyniku pojedynczą wartość możemy użyć np. w wyrażeniach warunkowych (`WHERE`, `HAVING`). 
> 
> Mając zapytanie o średnią liczbę wypożyczeń dokonanych przez użytkownika, możemy napisać zapytanie zwracające listę użytkowników pożyczających więcej książek niż średnia.

In [ ]:
%%sql
SELECT COUNT(DISTINCT borrowings.ID) / COUNT(DISTINCT borrowings.User_ID) FROM borrowings;

In [ ]:
%%sql
SELECT users.Name, users.Surname, COUNT(borrowings.ID) AS Books_borrowed
FROM users JOIN borrowings ON users.ID = borrowings.User_ID GROUP BY users.ID
HAVING books_borrowed > 8;

In [ ]:
%%sql
SELECT users.Name, users.Surname, COUNT(borrowings.ID) AS Books_borrowed
FROM users JOIN borrowings  ON users.ID = borrowings.User_ID GROUP BY users.ID
HAVING books_borrowed > (
    SELECT COUNT(DISTINCT borrowings.ID) / COUNT(DISTINCT borrowings.User_ID) FROM borrowings
);

### Podzapytania w `SELECT`

> **Przykład:**
>
> Analogicznie do poprzedniego przypadku - jeśli podzapytanie zwraca pojedynczą wartość, możemy je użyć jako jedną z kolumn w głównym zapytaniu.
>
> Co ważne - w podzapytaniu (w tym przypadku z tabeli `borrowings`) możemy odwołać się do tabeli i pól z zapytania zewnętrznego (`users`).

In [ ]:
%%sql
SELECT users.Name, users.Surname, 
(
    SELECT COUNT(borrowings.ID) FROM borrowings WHERE borrowings.User_ID = users.ID
) 
AS Books FROM users;

### Wydajność

Nie musimy się martwić o wydajność takich podzapytań - mimo że pozornie liczenie średniej liczby wypożyczeń powinno się wykonywać z osobna dla każdego rekordu użytkownika, baza danych optymalizuje plan zapytania. Jeśli chcemy sprawdzić, jak nasze zapytanie przekłada się na operacje wykonywane w bazie, możemy przed treścią zapytania dodać komendę `EXPLAIN QUERY PLAN` - zapytanie nie zostanie wtedy wykonane, otrzymamy jednak listę kroków, które baza danych chce wykonać.

> **Przykład:**
> 
> Zapytanie łączy tabelę `users` z `borrowings`, grupuje wyniki według użytkownika i filtruje tych, którzy wypożyczyli więcej książek niż średnia liczba wypożyczeń na użytkownika. `EXPLAIN QUERY PLAN` pokazuje, że SQLite wykonuje skanowanie tabeli `users`, a następnie używa **filtra Bloom**, który jest strukturą danych pozwalającą szybko sprawdzić, czy dany element może należeć do zbioru (kosztem rzadkich fałszywych trafień). Dzięki temu liczba sprawdzanych rekordów w `borrowings` jest mniejsza. Dla grupowania SQLite tworzy **tymczasowe B-drzewo**, czyli zbalansowaną strukturę indeksową optymalizującą operacje wyszukiwania i sortowania. Podzapytanie liczące średnią wypożyczeń również korzysta z **tymczasowego B-drzewa**, aby efektywnie przetwarzać unikalne wartości.

In [ ]:
%%sql
EXPLAIN QUERY PLAN 
SELECT users.Name, users.Surname, COUNT(borrowings.ID) AS Books_borrowed
FROM users JOIN borrowings ON users.ID = borrowings.User_ID GROUP BY users.ID
HAVING Books_borrowed > (
    SELECT COUNT(DISTINCT borrowings.ID) / count(DISTINCT borrowings.User_ID) FROM borrowings
);

## Zadania utrwalające

[`02c - Zaawansowane operacje w SQL – stringi, daty, aliasy, joins i podzapytania - zadania.md`](https://github.com/miklesz/SQL/blob/master/02_joins.md)